In [1]:
#initializing my project and entity in wandb
#Deep Learning Assignment 1
#CS23M064

!pip install wandb
import wandb
from wandb.keras import WandbCallback
wandb.init(project='Deep Learning Assignment 1', entity='CS23M064')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
#import numpy ,fashion_mnist and load data

from keras.datasets import fashion_mnist
import numpy as np

((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [3]:
#example image from each class

images = []
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for class_label in range(len(classes)):
    index = next(i for i, y in enumerate(y_train) if y == class_label)
    images.append(wandb.Image(x_train[index], caption=classes[class_label]))

wandb.log({"example image from each class": images})


In [4]:
class SingleLayer:

# optimizer and activation functions and initializing Momentum and velocity weights and bias
  def __init__(self, idim, nof_nodes, activation='', optimizer='gradient_descent', weight_type='random'):
    self.optimizer = self.do_optimizer(optimizer)
    self.opt=optimizer
    self.activation, self.activationForwardFunction, self.activationBackwardFunction = self.do_activation(activation)

    self.weights, self.bias = self.initialize(idim, nof_nodes, activation, weight_type=weight_type)
    self.pv_weight, self.pv_bias = np.zeros([nof_nodes, idim]), np.zeros([nof_nodes, 1])
    self.pm_weight, self.pm_bias = np.zeros([nof_nodes, idim]), np.zeros([nof_nodes, 1])
# initialization of weights and bias to layer with random-normal distribution or xavier distribution
  def initialize(self, nof_inputfeatures, nof_nodes,activation,weight_type):
    np.random.seed(1)
    if weight_type == 'random':
      w=np.random.normal(0.0,0.5,size=(nof_nodes, nof_inputfeatures))
    else:
      x=np.sqrt(nof_nodes)
      w=np.random.uniform(-(1/x), (1/x), size=(nof_nodes, nof_inputfeatures))
    b = np.ones([nof_nodes, 1])
    return w,b
# selection of optimizer based on input value
  def do_optimizer(self, optimizer):
    if optimizer == 'gradient_descent':
        return self.gradient_descent
    elif optimizer == 'momentum_gradient_descent':
        return self.momentum_gradient_descent
    elif optimizer == 'rmsprop':
        return self.rmsprop
    elif optimizer == 'adam':
        return self.adam
    elif optimizer == 'stochastic_gradient_descent':
        return self.stochastic_gradient_descent
    elif optimizer == 'nadam':
        return self.nadam
    elif optimizer == 'nesterov':
        return self.nesterov
#selection of activation function based on input value
  def do_activation(self, activation):
    if activation == 'sigmoid':
        return activation, self.sigmoid, self.sigmoid_grad
    elif activation == 'relu':
        return activation, self.relu, self.relu_grad
    elif activation == 'tanh':
        return activation, self.tanh, self.tanh_grad
    else:
        return 'softmax', self.softmax, self.softmax_grad

#activation functions and their derivatives
#sigmoid function
  def sigmoid(self, Z):
    Z=np.clip(Z,500,-500)
    A = 1 / (1 + np.exp(-Z))
    return A

# derivative of sigmoid function
  def sigmoid_grad(self, derivative_A):
    e=np.exp(-self.previous_Z)
    s = 1/(1+e)
    derivative_Z = derivative_A * s * (1 - s)
    return derivative_Z

 #tanh activation function
  def tanh(self,Z):
    return np.tanh(Z)
# derivative of tanh function
  def tanh_grad(self,derivative_A):
    s=self.tanh(self.previous_Z)
    ss=(s**2)
    return derivative_A*(1-ss)
#relu activation function
  def relu(self,Z):
    A= np.maximum(0,Z)
    return A
# derivative of relu function
  def relu_grad(self,derivative_A):
    s= np.maximum(0,self.previous_Z)
    t = 1.*(s>0)*derivative_A
    return t
#softmax activation function
  def softmax(self,Z):
    maxZ=np.max(Z)
    eZ=np.exp(Z - maxZ)
    A = eZ/eZ.sum(axis=0, keepdims=True)
    return A
#gradient of softmax function
  def softmax_grad(self,derivative_A):
    return derivative_A

#forward propagation of input vector A
  def forward_propagate(self, A):
    if self.opt != 'nesterov':
      Z= np.dot(self.weights,A) + self.bias
    else:
      xw=0.9*self.pv_weight
      xw=self.weights-xw
      xb=0.9*self.pv_bias
      xb=self.bias-xb
      Z=np.dot(xw, A) + xb
    self.previous_A = A
    self.previous_Z = Z
    A = self.activationForwardFunction(Z)
    return A
#backward propagation
  def backward_propagate(self, derivative_A):
    sp=self.previous_A.shape[1]
    derivative_Z = self.activationBackwardFunction(derivative_A)
    sum_value=np.sum(derivative_Z, axis=1, keepdims=True)
    self.derivative_b = 1/sp*sum_value
    self.derivative_w = 1 / sp * np.dot(derivative_Z, self.previous_A.T)
    return np.dot(self.weights.T, derivative_A)

  def predict(self,A):
    x=np.dot(self.weights,A)
    Z=self.bias + x
    A=self.activationForwardFunction(Z)
    return A

#stochastic gradient descent algorithm for updating weights
  def stochastic_gradient_descent(self, derivative_A,learn_rate = 0.001,t = 0,l2_lambda=0,batch_size = 32):
    derivative_Z=self.activationBackwardFunction(derivative_A)
    a= self.previousA.shape[1]
    previous_derivative_A = np.dot(self.weights.T, derivative_Z)
    for i in range(a):
      b=derivative_Z[:,i:i+1]
      self.derivative_b=1/a*b
      self.derivative_w = 1/a*np.dot(b,self.previous_A[:,i:i+1].T)
      c=l2_lambda/batch_size
      self.weights -=(learn_rate * self.derivative_w)- learn_rate *c*self.weights
      self.bias -= (learn_rate * self.derivative_b) - c*self.bias
    return previous_derivative_A

    # rmsprop algorithm for gradient descent
  def rmsprop(self, learn_rate,t,l2_lambda=0,batch_size =32, mrate = 0.9):
    gws=np.square(self.derivative_w)
    gbs = np.square(self.derivative_b)
    nmrate=1-mrate
    self.pv_weight = mrate * self.pv_weight + nmrate * gws
    self.pv_bias = mrate * self.pv_bias + nmrate *gbs
    self.pv_bias[self.pv_bias<0] = 1e-9
    for i in self.pv_weight:
      i[i<0]=1e-9
    a= (learn_rate * l2_lambda / batch_size)
    self.weights=self.weights-a * self.weights
    self.bias= self.bias -a * self.bias
    b=np.sqrt(self.pv_bias+(1e-8))
    c= learn_rate/b
    self.weights = self.weights - c*self.derivative_w
    self.bias = self.bias - c*self.derivative_b

  def gradient_descent(self, learn_rate,l2_lambda =0,batch_size =32,t=0):
    c=l2_lambda/batch_size
    self.weights = self.weights - learn_rate * self.derivative_w-learn_rate*c*self.weights
    self.bias = self.bias - learn_rate * self.derivative_b-c*self.bias

 #momentum gradient descent
  def momentum_gradient_descent(self, learn_rate,t,l2_lambda=0,batch_size =32, mrate=0.9):
    c=l2_lambda/batch_size
    self.pm_weight= mrate*self.pm_weight + learn_rate * self.derivative_w+c*self.weights
    self.pm_bias= mrate * self.pm_bias+learn_rate*self.derivative_b+c*self.bias
    self.weights-=self.pm_weight
    self.bias -=self.pm_bias

#nesterov algorithm
  def nesterov(self,learn_rate,mrate = 0.9,l2_lambda =0,batch_size =32,t = 0):
    self.pv_weight = mrate * self.pv_weight + learn_rate * self.derivative_w
    self.weights-=self.pv_weight
    self.bl = self.bias - mrate * self.pv_bias
    self.pv_bias *= mrate

#adam algorithm
  def adam(self,learn_rate , beta1 = 0.9, beta2 = 0.999,l2_lambda =0,batch_size =32,t=0):
    nbeta1=1-beta1;
    nbeta2=1-beta2;
    self.pm_weight = beta1 * self.pm_weight + nbeta1*self.derivative_w
    self.pm_bias = beta1 * self.pm_bias + nbeta1*self.derivative_b
    sw=np.square(self.derivative_w)
    sb=np.square(self.derivative_b)
    self.pv_weight = beta2 * self.pv_weight+ nbeta2*sw
    self.pv_bias = beta2 * self.pv_bias + nbeta2*sb
    self.pm_weightH = self.pm_weight/nbeta1
    self.pm_biasH = self.pm_bias/nbeta1
    self.pv_weightH = self.pv_weight/nbeta2
    self.pv_biasH = self.pv_bias/nbeta2
    rw=np.sqrt(self.pv_weightH+(1e-8))
    self.weights = self.weights - learn_rate * np.divide(self.pm_weightH,rw)
    rb=np.sqrt(self.pv_biasH+(1e-8))
    self.bias = self.bias - learn_rate * np.divide(self.pm_biasH,rb)



#nadam algorithm
  def nadam(self,learn_rate ,t, beta1 = 0.9, beta2 = 0.999,l2_lambda =0,batch_size =32):
    nbeta1=1-beta1
    nbeta2=1-beta2
    self.pm_weight=beta1*self.pm_weight+nbeta1*self.derivative_w
    self.pm_bias=beta1*self.pm_bias+nbeta1*self.derivative_b
    sw=np.square(self.derivative_w)
    sb=np.square(self.derivative_b)
    self.pv_weight=beta2*self.pv_weight+nbeta2*sw
    self.pv_bias=beta2*self.pv_bias+nbeta2*sb
    self.pm_weightH = (beta1 * self.pm_weight /nbeta1) + self.derivative_w
    self.pm_biasH = (beta1 * self.pm_bias /nbeta1) + self.derivative_b
    self.pv_weightH = (beta2 * self.pv_weight) / nbeta2
    self.pv_biasH = (beta2 * self.pv_bias) / nbeta2
    sw=np.sqrt(self.pv_weightH+(1e-8))
    aw=np.divide(self.pm_weightH,sw)
    sb= np.sqrt(self.pv_biasH+(1e-8))
    ab= np.divide(self.pm_biasH,sb)
    self.weights -= (learn_rate *aw)
    self.bias -= (learn_rate *ab)



In [5]:



class FeedForwardNeuralNetwork:

    def __init__(self, layers_size,epochs=5,learn_rate=0.001, l2_lambda = 0,optimizer = 'gradient_descent', activation = 'sigmoid',weight_type = 'random', loss='cross_entropy'):
        self.layers=[]
        self.layers_size = layers_size
        self.epochs = epochs
        self.learn_rate = learn_rate
        self.optimizer = optimizer
        self.activation = activation
        self.weight_type = weight_type
        self.l2_lambda = l2_lambda
        if loss =='mean_square':
            self.losscomputation = self.mean_quare
            self.lossBackwardpass = self.mean_square_grad
        elif loss =='cross_entropy':
            self.losscomputation = self.cross_entropy
            self.lossBackwardpass = self.cross_entropy_grad
        else:
            print('loss computation is invalid')
        self.loss=loss

  # addition of  layer to the feedforward neural network
    def addingLayer(self, idim=None, nof_nodes=1, activation='', weight_type='random'):
        if not self.layers:
            if idim is None:
              print('Invalid number of layers')
        else:
            if idim is None:
              idim = self.layers[-1].outputDimension()
        add_layer = SingleLayer(idim,nof_nodes, activation, optimizer=self.optimizer, weight_type=weight_type)
        self.layers.append(add_layer)

    # mean_square error
    def mean_square(self, Y, A):
        l=np.square(Y - A)
        b_sum=np.sum(l)
        a= Y.shape[1]
        c = 1 / a * b_sum
        return np.squeeze(c)

    # mean_square_grad
    def mean_square_grad(self, Y, A):
        x=Y-A
        dA = -2 * x
        return dA


    # cross_entropy
    def cross_entropy(self, Y, A):
        a = Y.shape[1]
        b=np.sum(Y*np.log(A))
        c = -(1/a)*b
        return np.squeeze(c)


    # cross_entropy_grad
    def cross_entropy_grad(self, Y, A):
        dA = A-Y
        return dA


    # to get loss from predicted values
    def cost(self, Y, A):
        return self.losscomputation(Y, A)


    # Forwarding X through all layers
    def forward_propagate(self, X):
        result = np.copy(X)
        for each_layer in self.layers:
            result = each_layer.forward_propagate(result)
        return result


    # Backward pass Y and A in reverse direction
    def backward_propagate(self, Y, A):
        derivative_A = self.lossBackwardpass(Y, A)
        if self.optimizer != 'stochastic_gradient_descent':
            for each_layer in reversed(self.layers):
                derivative_A = each_layer.backward_propagate(derivative_A)
        elif self.optimizer =='stochastic_gradient_descent':
            for each_layer in reversed(self.layers):
                derivative_A = each_layer.stochastic_gradient_descent(derivative_A,learn_rate = self.learn_rate)


    # Update the weights and calculate gradient descent of all the layers
    def update_Weight(self, learn_rate=0.01,l2_lambda =0,batch_size=32,t=0):
        for each_layer in self.layers:
            each_layer.optimizer(learn_rate,l2_lambda = l2_lambda,batch_size = batch_size,t=0)


    # Train data for validation and test data
    def fit(self,x_train,y_train,x_test,y_test,batch_size = 32):

        from sklearn.model_selection import train_test_split

        x,x_value,y,y_value = train_test_split(x_train,y_train,train_size = 0.9, test_size = 0.1, random_state=10)

        if self.activation=='relu':
          self.weight_type = 'xavier'
    #add the layers with activation functions by calling adding layer
        l=len(self.layers_size)
        for k in range(1,l-1):
          self.addingLayer(idim=self.layers_size[k-1], nof_nodes=self.layers_size[k], activation=self.activation, weight_type = self.weight_type)


    # softmax activation function for the last l layer
        self.addingLayer(idim=self.layers_size[-2], nof_nodes=self.layers_size[-1], activation='softmax', weight_type = self.weight_type)

    #one hot encoder
        i=len(y)
        j=len(set(y))
        y_encoder = np.zeros([j,i])
        for k in range(y_encoder.shape[1]):
          y_encoder[y[k]][k] = 1

  #Train the data
        for i in range(self.epochs):

  #avoid gradient vanishing
          if self.activation =='relu':
            if self.optimizer == 'momentum_gradient_descent'  or self.optimizer == 'nesterov'or self.optimizer == 'rmsprop':
              d=self.learn_rate/15
              self.learn_rate=d
          #Training the data for each batch
          for k in range(0,x.shape[0],batch_size):
            kbatch=k+batch_size
            xbatch = x[k:kbatch]
            ybatch = y[k:kbatch]
            y_encoderbatch = y_encoder[:,k:kbatch]
            xbatch = xbatch.reshape(xbatch.shape[0],xbatch.shape[1]*xbatch.shape[2]).T
            mi=np.min(xbatch)
            mx=np.max(xbatch)
            xbatch = xbatch-mi/mx-mi
            #feed forward neural network
            if self.optimizer == 'stochastic_gradient_descent':
              A = self.forward_propagate(xbatch)
              self.backward_propagate(y_encoderbatch,A)
            elif self.optimizer != 'stochastic_gradient_descent':
              A = self.forward_propagate(xbatch)
              self.backward_propagate(y_encoderbatch,A)
              self.update_Weight(learn_rate=self.learn_rate,l2_lambda = self.l2_lambda,batch_size=batch_size,t= i+1)

          #Predicting the Loss, Accuracy and Predicted labels for Validation data and Test data
          validation_loss,validation_acc,_=self.predict(x_value,y_value)
          loss,accuracy,y_pred= self.predict(x_test,y_test)

          #Displaying The Loss and Accuracy for Validation data and Test data
          print("After ",i+1,"iterations:")
          print("validation loss;",validation_loss,"validation accuracy:",validation_acc)
          print("test_loss:",loss,"test accuracy:",accuracy)

          #Logging the loss and accuracy values for each epoch in wandb panel
          wandb.log({"val_loss":validation_loss,"val_accuracy":validation_acc,"loss":loss,"accuracy":accuracy,"epoch":i})

        return y_pred #returning the probabilistic distributions of each class


    #predficting loss and accuracy
    def predict(self,x,y):
        i=x.shape[0]
        j=x.shape[1]*x.shape[2]
        A = x.reshape(i,j).T
        a=len(set(y))
        b=len(y)
        y_encoder = np.zeros([a,b])
        for k in range(y_encoder.shape[1]):
          y_encoder[y[k]][k] = 1

        for each_layer in self.layers:
            A = each_layer.predict(A)
        x=-(y_encoder * np.log(A))
        cross_entropy = x.mean() * y_encoder.shape[0]
        y_pred = np.argmax(A,axis = 0)
        accuracy = (y==y_pred).mean()

        return cross_entropy,accuracy,A


In [6]:
#Sweep Configuration with Bayesian Strategy

sweep_config = {
    'name':"my-sweep",
    'method': 'bayes',
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'
    },

    'parameters': {
        'epochs': {
            'values': [5, 10] #number of epochs
        },
        'hidden_layer': {
            'values': [3, 4, 5] #number of hidden layers
        },
        'hidden_size': {
            'values':[32, 64, 128] #size of every hidden layer
        },
        'weight_decay': {
            'values':[0, 0.0005,  0.5] #L2 regularisation
        },
        'learn_rate': {
            'values': [1e-3, 1e-4]
        },
        'optimizer': {
            'values': ['momentum_gradient_descent', 'nesterov', 'rmsprop', 'adam', 'nadam','stochastic_gradient_descent']
        },
        'batch_size' : {
            'values':[16, 32, 64]
        },
        'weight_initial': {
            'values':['random','xavier']
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        }

        }
}

sweep_id = wandb.sweep(sweep_config, entity="CS23M064", project="Deep Learning Assignment 1")

Create sweep with ID: zkbm0kqa
Sweep URL: https://wandb.ai/cs23m064/Deep%20Learning%20Assignment%201/sweeps/zkbm0kqa


In [7]:
#defining train function for wandb agent to run it's configurations (directly from documentation)

def train():
  with wandb.init() as run:

    config = wandb.config

    wandb.run.name = "hl_" + str(config.hidden_size)+"_bs_"+str(config.batch_size)+"_ac_"+ config.activation
    np.random.seed(1)
    model = FeedForwardNeuralNetwork(layers_size = [784]+[config.hidden_size]*config.hidden_layer+[10],epochs = config["epochs"],learn_rate = config.learn_rate,l2_lambda = config.weight_decay,loss='cross_entropy',activation = config.activation, optimizer = config.optimizer, weight_type=config.weight_initial )
    y_pred = model.fit(x_train,y_train,x_test,y_test,batch_size=config.batch_size)

#running the sweep
wandb.agent('zkbm0kqa',train,entity="CS23M064", project="Deep Learning Assignment 1")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: jpt9i5nk with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layer: 5
wandb: 	hidden_size: 64
wandb: 	learn_rate: 0.0001
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.0005
wandb: 	weight_initial: random
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    sel

After  1 iterations:
validation loss; 0.8321116067089719 validation accuracy: 0.7311666666666666
test_loss: 0.8291160031776392 test accuracy: 0.7372
After  2 iterations:
validation loss; 0.7449074628244029 validation accuracy: 0.755
test_loss: 0.7521094011116636 test accuracy: 0.7601
After  3 iterations:
validation loss; 0.6751735278663312 validation accuracy: 0.782
test_loss: 0.6947301121576464 test accuracy: 0.7791
After  4 iterations:
validation loss; 0.6285540715140909 validation accuracy: 0.7933333333333333
test_loss: 0.6468630994077135 test accuracy: 0.7895
After  5 iterations:
validation loss; 0.5985327439868386 validation accuracy: 0.8033333333333333
test_loss: 0.6132720341937508 test accuracy: 0.8003


accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▆▄▂▁
val_accuracy,▁▃▆▇█
val_loss,█▅▃▂▁
accuracy,0.8003
epoch,4
loss,0.61327
val_accuracy,0.80333
val_loss,0.59853


wandb: Agent Starting Run: g0xsalnc with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layer: 4
wandb: 	hidden_size: 64
wandb: 	learn_rate: 0.0001
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.0005
wandb: 	weight_initial: xavier
wandb: Currently logged in as: cs23m064. Use `wandb login --relogin` to force relogin


After  1 iterations:
validation loss; 0.9533940909863675 validation accuracy: 0.6606666666666666
test_loss: 0.956063696581464 test accuracy: 0.6601
After  2 iterations:
validation loss; 0.7761626511472284 validation accuracy: 0.7083333333333334
test_loss: 0.7743978831269814 test accuracy: 0.7095
After  3 iterations:
validation loss; 0.7261324227047977 validation accuracy: 0.7193333333333334
test_loss: 0.7335950395004798 test accuracy: 0.722
After  4 iterations:
validation loss; 0.6918815068763612 validation accuracy: 0.7338333333333333
test_loss: 0.6998292401059305 test accuracy: 0.7339
After  5 iterations:
validation loss; 0.6660456339447557 validation accuracy: 0.7433333333333333
test_loss: 0.6758245056282006 test accuracy: 0.7452
After  6 iterations:
validation loss; 0.652185805180028 validation accuracy: 0.7518333333333334
test_loss: 0.6649113319589608 test accuracy: 0.7494
After  7 iterations:
validation loss; 0.639539400555732 validation accuracy: 0.7598333333333334
test_loss: 0.

...

After  1 iterations:
validation loss; 2.3026015236146273 validation accuracy: 0.0955
test_loss: 2.302585256490609 test accuracy: 0.1
After  2 iterations:
validation loss; 2.3026128560399957 validation accuracy: 0.0955
test_loss: 2.302585543689753 test accuracy: 0.1
After  3 iterations:
validation loss; 2.3026202781530136 validation accuracy: 0.0955
test_loss: 2.302585806844056 test accuracy: 0.1
After  4 iterations:
validation loss; 2.302625121052354 validation accuracy: 0.0955
test_loss: 2.3025860097569866 test accuracy: 0.1
After  5 iterations:
validation loss; 2.302628273466526 validation accuracy: 0.0955
test_loss: 2.3025861548616473 test accuracy: 0.1
After  6 iterations:
validation loss; 2.302630322329645 validation accuracy: 0.0955
test_loss: 2.302586254616185 test accuracy: 0.1
After  7 iterations:
validation loss; 2.302631652637152 validation accuracy: 0.0955
test_loss: 2.302586321666824 test accuracy: 0.1
After  8 iterations:
validation loss; 2.3026325158404854 validation acc

accuracy,▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,▁▃▅▆▆█████
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▃▅▆▇▇████
accuracy,0.1
epoch,9
loss,2.30259
val_accuracy,0.0955
val_loss,2.30263


wandb: Agent Starting Run: topih3i7 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_layer: 4
wandb: 	hidden_size: 128
wandb: 	learn_rate: 0.0001
wandb: 	optimizer: stochastic_gradient_descent
wandb: 	weight_decay: 0.0005
wandb: 	weight_initial: random
wandb: Ctrl + C detected. Stopping sweep.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7dd952d94b80>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe